In [1]:
# Hier werden alle Bibliotheken impotriert welche für das Programm benutzt werden. 
# Dies wird in der ersten Zelle gemacht, damit man sie nicht mehrmals laden muss
# Nach jeder Library werde ich noch den Link dazu angeben

import numpy as np 
#https://numpy.org
import torch 
# https://pytorch.org
import torch.nn as nn
from torch.utils.data import random_split
from torch.utils.data import Dataset
from torch.utils.data import Subset
import torchvision.datasets as datasets
import torchvision.transforms as transforms 

import glob
# https://docs.python.org/3/library/glob.html
import os
# https://docs.python.org/3/library/os.html
from sklearn.model_selection import train_test_split
# https://scikit-learn.org/stable/
from collections import Counter
# https://docs.python.org/3/library/collections.html
from itertools import zip_longest
# https://docs.python.org/3/library/itertools.html

In [2]:
# Dies ist die Lernrate, welche wichtig für die "Schrittgrösse des Programmes" ist
learning_rate= 0.0000001

# Hier ist die Funktion für das Laden der Daten
# Diese Funktion nimmt alle Bilddaten und die Klassen und steckt diese in einen Datensatz 
# Dieser Datensatz hat dann das Label und die Daten in einem
# Diese Daten müssen durch iterationen durch diesen herausgelesen werden
class MyDataset(Dataset):
            def __init__(self, images, labels):
                self.images = images
                self.labels = labels

            def __getitem__(self, index):
                image = self.images[index]
                label = self.labels[index]
                return image, label

            def __len__(self):
                return len(self.images)

# Definiere das CNN (genauere Erklärung ist weiter oben in dem Dokument)
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()

        # Definiere die Convolutional Schichten
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)

        # Definiere die  max pooling Schichten
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Definiere die fully connected Schichten
        self.fc1 = nn.Linear(in_features=576, out_features=128) #64, 128
        self.fc2 = nn.Linear(in_features=128, out_features=num_classes)

        # Defniere die voll axtiven Funktionen
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        #Alle funktionen anwenden
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool(x)

        # Den output verflachen
        x = x.view(x.size(0), -1)
        
        # Die voll-verbunden Schichten anwenden
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        # Die softmax funktion anwenden
        x = self.softmax(x)

        return x



# Hier wird der Weg zu den Daten definiert, dieser Variirt mit jedem Nutzer
# In dem Ordner, den man dort findet hat es noch verschiedene Unterordner
# In diesen Unterordner sind die Daten zu finden
folder_path = r'C:\Users\41793\Desktop\Programming\BG_Creative_Coding\rnn_tutorial_npydata_subfolder2'

# Hier iterieren wir durch die verschidenen Ordnern
# Der ganze nachfolgende Code wird pro Ordner einmal ausgefürt
# In meinem Fall hat es in diesem Ordner 35 Unterordner mit jeweils 10 Klassen, mit jeweils ungefähr 150'000 Bildern
# Dies bedeutet, dass hier der untenstehende Code 35 mal abgerufen wird
for folder in os.listdir(folder_path):
    
    # Dies ist reine kosmetik, so dass ich besser sehen kann bei welchem Ordner wir sind
    # Nach dieser Ausgabe des Ordners in welchem wir und befinden, folgen dann später die Genauigkeiten des Programmes
    print("*********")
    print(folder)
    print("*********")
    
    
    # Hier erhalten wir den kompletten Weg zu dem Ordner in welchem sich jetzt die 10 Klassen befinden
    file_path = os.path.join(folder_path, folder)
    
    # Hier sehen wir ob der obige ORdnerweg ein weiterer Ordner hat
    if os.path.isdir(file_path):
        # Falls dies der Fall ist, soll durch diese Daten durch iteriert werden
        for subfile in os.listdir(file_path):
            # Hier erhalten wir den vollen Weg zu jenem Unterordner
            subfile_path = os.path.join(file_path, subfile)
            
        # Jetzt sagen wir, dass die Daten welche in diesem Ordner sind files heissen, solange sie mit .npy enden
        # Hier nutzen wir die glob Library welche es uns ermöglicht einfacher auf Ordner auf underem Gerät zuzugreifen
        files = glob.glob(os.path.join(file_path, "*.npy"))

        # Nun setzten wir leere Listen für die Daten und Labels fest
        data_list = []
        label_list = []
        label_list_alone = []
        i = 0
        # Hier gehe wir über die verschiedenen Dateien in Dateien
        for file in files:
            # Nun legen wir das Label fest
            # Da dies noch nicht in den Daten vorgegeben wurden, nehmen wir einfach den Namen der Datei
            # Dies erreichen wir wieder mit der glob Library, indem wir einfach nur den Basename nehmen
            # Dieser Basename ist somit der einfache Name der Datei, ohne alle Ordner, welche davor noch kämen
            # Wir spalten also den String bei diesem Basename und nehmen dann nur den teil in welchem der Name der Datei ist
            label = os.path.splitext(os.path.basename(file))[0]

            # Hier öffnen wir die Daten in Binarymode
            # Wir können die Daten nur mit dem rb = read in binary öffnen
            # Dies ist der Fall, da man nicht wie normal die Daten mit r laden kann, wegen ihres Formates
            # Passen sie hier auf, dass sie die Daten nicht mit wb öffnen,da die Daten sonst automatisch überschrieben werden
            # In underem Fall wäre dies ungünstig, da wir nichts reinschreiben und die daten somit leer werden
            with open(file, 'rb') as fin:
                # Nun werden die Daten mit der Library Numpy geladen
                data = np.load(fin)
                # Da wir nur ein Label pro Datei haben müssen wir dieses vervielfätigen
                # Falls wir dies nicht tun, haben wir das Problem, dass die Daten-liste und Label-Liste nicht gleich lang ist
                # Wir lösen dies, indem wir einfach für die länge der Daten das Label der Label-Liste hinzufügen
                for i in range (0, len(data)):
                    label_list.append(label)
            # Hier fügen wir nun auch die Daten der Daten-Liste hinzu
            data_list.append(data)
         
        # Dies ist die Grösse, mit welcher die Daten geladen werden
        batch_size = 100

        # Concatenate data and labels into arrays
        # Nun verketten wir die Daten und Labels zu Reihen, hier Array genannt
        # Die Daten werden hier auf eine Achse beschränkt, damit sie die gleichen Dimensionen wie die Lables haben
        data = np.concatenate(data_list, axis=0)
        labels = np.array(label_list)

        # Jetzt machen wir aus den Daten Kommazahlen, da eine spätere Funktionen nicht mit Integerns funktioniert
        data = data.astype(float)
        
        # Hier machen wir ein Wörterbuch für die Lables
        # Dies wird hier benötigt, da mein CNN nur Zahlen als Input nehmen kann
        # Das heisst, dass die Lables auch in Zahlenform sein müssen, damit eine Evaluation statt finden kann
        # Dieses Wörtbuch hat nun zumbeispiel den Eintrag: "Auto = 1, Biene = 2, etc."
        label_dict = {label: i for i, label in enumerate(set(labels))}

        # Convert labels to integers using the dictionary
        # Nun verwandeln wir die Lables, welche noch Strings sind, zu Integers
        # Dies erreichen wir, indem wir das label_dict verwenden
        labels = np.array([label_dict[label] for label in labels])

        # Dies ist mehr eine absicherung, dass auch wirklch alles eine Array ist
        data = np.array(data)
        labels = np.array(labels)

        # Hier sehen wir definieren wir, ob die Daten farbig oder schwarz, weiss sind
        # Falls num_channels = 3 sind, dann wäre es farbig, aufgrund der drei rgb farben
        # Falls jedoch, wie in unserem Fall dies 1 beträgt sind die Daten Schwarz-Weiss
        num_channels = data.ndim - 1
        
        """
        # Hier könnte man noch einige Bilder ausgebe lassen, falls man eine Idee haben will, wie die Daten aussehen
        import matplotlib.pyplot as plt

        for i in range(1):
            sample = data[i]
            print(f'Sample {i}: {sample}')

        # Wir nehmen an, das die Daten eine 2D Reihe sind, mit Form (höche, breite)
            image = np.array(sample)
            image = image.reshape(28, 28)

            # Zeig das Bild
            plt.imshow(image)
            plt.show()
            """

        # Hier sagen wir, das 80% der Batchsize die Input Batchsize ist
        # Dies wirdvnoch später verwendet, um die Menge der Daten zu bestimmen
        input_batch = int(0.8* batch_size)

        # Diese Funktion hätte direkt im CNN defniert werden können
        # Ich habe sie hier definiert, da ich es erst während des schon laufenden Programmes bemerkte
        # Es mach keinen unterschied wo es definiert wird
        output_features = 24 * 24

        # Wähend des Testes hatte ich mehr mals den Fehler wegen des Types
        # Wegen dem habe ich die Daten hier noch einmal, ist jedoch nicht nötig
        # Da dieser Code hier nur 35 Mal ablauft, hat dies keinen grossen einfluss auf die Effizienz
        data = np.array(data)
        labels = np.array(labels)
        
        # Hier würden die Daten auf der CPU geladen werden, falls die Grafikkarte zu schwach wäre, oder man keine hat
        #data = data.cpu().numpy()
        #labels = labels.cpu().numpy()

        # Create a dataset from your data and labels
        # Nun kreiiren wir ein Datenset mit unseren Daten und Labels
        # Man greift auf die zu Beginn defnierte Funktion MyDataset zurück 
        dataset = MyDataset(data, labels)

        # Hier definieren wir noch einige notwendige Variablen für den Rest des Programmes 
        input_size = data.shape[1]
        hidden_size = 128
        num_layers = 1
        num_classes = len(label_dict)
        batch_size = batch_size  

        # Jetzt wird das CNN inizialiesiert, dass heisst geladen
        # Die num_classes sind die Anzahl an Klassen, welche wir haben, dies kann aus dem Wörterbuch herausgelsesen werden
        cnn = CNN(num_classes=num_classes)

        # Da das CNN in unserem Fall sehr viel Nechenleistung beanspruch müssen wir es wenn möglich verschieben
        # Dies erreichen wir, indem wir mit torch.device nachschauen was vorhanden ist
        # Für den Fall, das eine Grafikkarte vorhanden ist und Cuda unterstützt wird, ist device = GPU
        # Dies ist stark zu empfehlen, da die Rechenzeit sonst viel zu hoch ist
        # Für den Fall das keine GPU vorhande ist, wird einfach die CPU verwendet
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Hier wird das CNN zu dem verfügbaren Gerät verschoben
        cnn.to(device)

        # Nun definieren wir die Loss und Optimizer funktionen
        # Diese sind essenziell für das CNN, da es sonst nicht lernen könnte
        # Die CrossEntropyLoss Funktion ist eine häufig verwendete Verlust-Funktion
        # Es misst den unterschied zischen der Vorhergesagten Verteilung der Klassen und der realen Verteilung
        # Dies wird vereinfaacht berechnet, in dem man den unterschied nehmen und diesen dann mit -y * ln(y) multiplizieren
        # Hier wird dies ausführlich erklärt: https://vitalflux.com/mean-squared-error-vs-cross-entropy-loss-function/
        criterion = nn.CrossEntropyLoss()
        # Nun haben wir hier den optimizer
        # Hier wurde der Adam optimizer gewählt, welcher einer der häufigsten ist
        # Die Wahl wurde auf Grund von Tests genommen
        # In einem CNN (Convolutional Neural Network) braucht man einen Optimierer, um die Gewichte des Netzwerks zu aktualisieren und zu optimieren. Der Optimierer ist dafür verantwortlich, den Fehler des Netzwerks zu minimieren, indem er die Gewichte des Netzwerks anpasst, um die Vorhersagen des Netzwerks so nah wie möglich an den tatsächlichen Ergebnissen zu bringen.
        # Ein Optimierer verwendet die Gradienten des Verlustes, die durch die Backpropagation berechnet werden, um die Gewichte des Netzwerks zu aktualisieren. Der Gradient beschreibt die Änderung des Verlustes in Bezug auf die Gewichte und die Optimierer nutzen diese Informationen, um die Gewichte zu aktualisieren und den Verlust zu minimieren.
        # Der Adam Optimierer kombiniert die Vorteile von zwei anderen Optimierern: dem Momentum Optimierer und dem AdaGrad Optimierer.
        # Der Momentum Optimierer hilft dabei, das Netzwerk schneller aus flachen Minima herauszubewegen und das AdaGrad Optimierer hilft dabei, die Lernrate für jeden Parameter anzupassen.
        # Adam nutzt diese zwei ansätze und ist somit populär
        # Frü eine genauere erklärung verweise ich gerne auf diesen Artikel: https://artemoppermann.com/de/optimierung-in-deep-learning-adagrad-rmsprop-adam/
        optimizer = torch.optim.Adam(cnn.parameters(), lr = learning_rate)

        test_batch = int(batch_size * 0.2)

        # Nun laden wir die Daten mit einer der vielen Funktionen von PyTorch
        # Diese Funktion erstellt einen Dataloader, der die Daten in Batches aufteilt und sie zufällig mischt (shuffle=True) und das letzte Batch verwirft (drop_last=True), wenn es nicht vollständig gefüllt ist.
        dataloader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True, drop_last=True)

        # Hier teile ich die Daten in Training und Test Daten
        train_set_size = int(len(dataset) * 0.8)
        test_set_size = len(dataset) - train_set_size

        # Nun definieren wir die Anzahl an Epochen für welches das CNN trainieren wird
        # Ich hatte diesen Wert meistens bei 125 um das Netz zu trainieren
        # Dies fürte dazu, dass ich meinen PC  für eine ganze Woche laufen lies
        # Die Trainingszweit betrug 150 Stunden, wobei die Resultate mit längerem lernen noch besser werden würden
        num_epochs = 150
        # Hier folgt eine for Schleife, welche für die Anzahl an Epochen ausgefürt wird
        for epoch in range(num_epochs):
            # Jetzt Iterieren wir über den Dataloader, der die Daten in Batches aufteilt. 
            for data, labels in dataloader:
                # Wir wandeln die Labels in den Datentyp LongTensor
                labels = labels.type(torch.LongTensor)
                # Hier verschieben wir die Daten dann auf die GPU durch die Funktion .cuda()
                data, labels = data.cuda(), labels.cuda()
                # Wir splitten dann die Daten in Trainings- und Testdaten mit dem train_test_split() von sklearn
                train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2)
                # Wir wandeln die Trainingsdaten in den Datentyp float um ändern die Form der Daten von (batch_size, 28, 28) in (batch_size, 1, 28, 28)
                train_data = train_data.to(dtype=torch.float)
                # Danach ändern wir die Form der Daten von (batch_size, 28, 28) in (batch_size, 1, 28, 28)
                train_data = train_data.view(-1, 28, 28)
                train_data = train_data.view(input_batch, 1, 28, 28)
                # Wir führen die Daten durch das CNN und speichern das Ergebnis in der Variablen "output".
                # Dies ist der Schritt, welcher bei weitem am meisten Fehelr verursacht hatte
                # Dies ist so, da der Code bis hier hin ohne Fehler gelaufen ist, jedoch dann immer ein Fehler in Bezug auf die Form hervor kam
                # Dieser Schritt braucht als einzige Variabel wahrscheinlich am meisten Zeit
                # Zu Grunde liegen die vielen Umwandlung welche in unserem CNN geschehen
                output = cnn(train_data)
                # Wir berechnen den Verlust (loss) mit der verwendeten Loss-Funktion (criterion) zwischen dem output des Netzes und den tatsächlichen Trainingslabels.
                # Dies geschieht einfach indem wir die zufor definierte Funktion aufrufen
                loss = criterion(output, train_labels)
                # Wir setzen die Gradienten auf Null (zero_grad()) 
                optimizer.zero_grad()
                # Nun führen einen Backpropagation Schritt aus (backward())
                loss.backward()
                # Zu letzt aktualisieren wir die Gewichte des Netzes mit dem verwendeten Optimierer (optimizer.step()).
                optimizer.step()
                
            # Wenn die aktuelle Epoche ein Vielfaches von 1 ist, geben wir den aktuellen Verlust aus.
            # Mit den Werten welche ich gewählt hatte, passiert dies alle 100 Sekunden
            if (epoch+1) % 1 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

                
        # Dieser Code stellt eine Schleife dar, die verwendet wird, um die Genauigkeit des trainierten CNNs auf einem Testdatensatz zu überprüfen
        # Nun setzen wir das CNN zu Evaluations-Modus, dies bedeutet, dass das CNN nicht trainiert wird
        cnn.eval() 
        # Hier schalten wir die gradient Optimisierung ab
        with torch.no_grad():
            # Nun deklarieren wir die Variabeln correct und total das erste mal
            # Wir setzen diese auf null, da dies der natürliche Ausgangspunkt ist
            correct = 0
            total = 0
            # Nun iterieren wir genau glecih wie oben durch die Daten und verwenden die Testdaten 
            # Hier trainieren wird das Model jedoch nicht, sondern schauen einfach, was die Gewichte ausgeben würden
            for data, labels in dataloader:
                labels = labels.type(torch.LongTensor)
                data, labels = data.cuda(), labels.cuda()
                train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2)
                test_data = test_data.to(dtype=torch.float)
                test_data = test_data.view(-1, 28, 28)
                test_data = test_data.view(test_batch, 1, 28, 28)
                # Hier werden die Daten wieder durch das CNN gegeben und es gibt uns somit die Vorhersage zurück
                outputs = cnn(test_data)
                _, predicted = torch.max(outputs.data, 1)

                # Berechne die Anzahl an richtigen Vorhersagen
                correct = (predicted == test_labels).sum().item()

                # Nun berechnen wir in Prozent wie genau unser Model ist
                accuracy = 100 * correct / test_labels.size(0)
            # Nun geben wir die Genauigkeit aus.
            # Dies geschieht am Ende jedes Ordners und dies ist auch die Genauigkeit mit welcher das Model neue Bilder richtig zuorden wird
            # Diese Genauigkeit beträgt bei meinem Training im Schnitt um die 75%
            print("The Accuracy of the CNN is:", accuracy)

        # Damit wir dieses Model wieder verwenden können, müssen wir die Gewichte des Models in einem .pt speichern
        # Um zu verhindern, dass wir die schon zu vor gespeicherten Werte überschreiben, nehmen wir die ändernde Variable folder in den Namen der Datei
        # Durch diesen Namen ist es auch einfacher wieder sicherzustellen zu welchem Subfolder die Gewichte gehören
        torch.save(cnn.state_dict(), 'cnn_{}.pt'.format(folder))
        
        # Mir steht nur eine gewisse Anzahl an Speicher zur verfügung
        # Auf Grund dessen, muss ich die Variabel data löschen
        # Diese Variabel beansprucht in meinem Fall 10 GB an RAM und auch noch alle 8 Gb VRAM meiner GPU
        del data


*********
18
*********
Epoch [1/150], Loss: 2.2212
Epoch [2/150], Loss: 2.0894
Epoch [3/150], Loss: 1.9825
Epoch [4/150], Loss: 1.9991
Epoch [5/150], Loss: 2.0197
Epoch [6/150], Loss: 1.9040
Epoch [7/150], Loss: 1.9860
Epoch [8/150], Loss: 1.9338
Epoch [9/150], Loss: 1.8420
Epoch [10/150], Loss: 1.8570
Epoch [11/150], Loss: 1.9230
Epoch [12/150], Loss: 1.9045
Epoch [13/150], Loss: 1.9388
Epoch [14/150], Loss: 1.9467
Epoch [15/150], Loss: 1.8992
Epoch [16/150], Loss: 1.9181
Epoch [17/150], Loss: 1.8802
Epoch [18/150], Loss: 1.9296
Epoch [19/150], Loss: 1.8619
Epoch [20/150], Loss: 1.8626
Epoch [21/150], Loss: 1.8520
Epoch [22/150], Loss: 1.8889
Epoch [23/150], Loss: 1.8285
Epoch [24/150], Loss: 1.7966
Epoch [25/150], Loss: 1.8710
Epoch [26/150], Loss: 1.8736
Epoch [27/150], Loss: 1.8283
Epoch [28/150], Loss: 1.8672
Epoch [29/150], Loss: 1.8199
Epoch [30/150], Loss: 1.8409
Epoch [31/150], Loss: 1.8093
Epoch [32/150], Loss: 1.8918
Epoch [33/150], Loss: 1.8785
Epoch [34/150], Loss: 1.7810


Epoch [129/150], Loss: 1.6679
Epoch [130/150], Loss: 1.6876
Epoch [131/150], Loss: 1.6919
Epoch [132/150], Loss: 1.6725
Epoch [133/150], Loss: 1.6373
Epoch [134/150], Loss: 1.6753
Epoch [135/150], Loss: 1.7261
Epoch [136/150], Loss: 1.7033
Epoch [137/150], Loss: 1.6987
Epoch [138/150], Loss: 1.6643
Epoch [139/150], Loss: 1.6089
Epoch [140/150], Loss: 1.7704
Epoch [141/150], Loss: 1.6828
Epoch [142/150], Loss: 1.6917
Epoch [143/150], Loss: 1.5522
Epoch [144/150], Loss: 1.6649
Epoch [145/150], Loss: 1.7156
Epoch [146/150], Loss: 1.6049
Epoch [147/150], Loss: 1.6658
Epoch [148/150], Loss: 1.6236
Epoch [149/150], Loss: 1.6820
Epoch [150/150], Loss: 1.7886
The Accuracy of the CNN is: 90.0
*********
20
*********
Epoch [1/150], Loss: 2.0788
Epoch [2/150], Loss: 2.0731
Epoch [3/150], Loss: 2.0099
Epoch [4/150], Loss: 1.9952
Epoch [5/150], Loss: 2.1189
Epoch [6/150], Loss: 1.9225
Epoch [7/150], Loss: 1.9941
Epoch [8/150], Loss: 2.0498
Epoch [9/150], Loss: 1.9164
Epoch [10/150], Loss: 2.0705
Epo

Epoch [106/150], Loss: 1.7429
Epoch [107/150], Loss: 1.8625
Epoch [108/150], Loss: 1.8184
Epoch [109/150], Loss: 1.8855
Epoch [110/150], Loss: 1.8776
Epoch [111/150], Loss: 1.8111
Epoch [112/150], Loss: 1.8131
Epoch [113/150], Loss: 1.9298
Epoch [114/150], Loss: 1.8346
Epoch [115/150], Loss: 1.8872
Epoch [116/150], Loss: 1.9533
Epoch [117/150], Loss: 1.9011
Epoch [118/150], Loss: 1.8985
Epoch [119/150], Loss: 1.8261
Epoch [120/150], Loss: 1.8799
Epoch [121/150], Loss: 1.8989
Epoch [122/150], Loss: 1.9373
Epoch [123/150], Loss: 1.8670
Epoch [124/150], Loss: 1.8616
Epoch [125/150], Loss: 1.8820
Epoch [126/150], Loss: 1.9909
Epoch [127/150], Loss: 1.8426
Epoch [128/150], Loss: 1.9103
Epoch [129/150], Loss: 1.9027
Epoch [130/150], Loss: 1.8828
Epoch [131/150], Loss: 1.8960
Epoch [132/150], Loss: 1.8713
Epoch [133/150], Loss: 1.8056
Epoch [134/150], Loss: 1.8921
Epoch [135/150], Loss: 1.8005
Epoch [136/150], Loss: 1.8980
Epoch [137/150], Loss: 1.8623
Epoch [138/150], Loss: 1.8775
Epoch [139

Epoch [82/150], Loss: 1.6839
Epoch [83/150], Loss: 1.6664
Epoch [84/150], Loss: 1.7200
Epoch [85/150], Loss: 1.7067
Epoch [86/150], Loss: 1.7408
Epoch [87/150], Loss: 1.7060
Epoch [88/150], Loss: 1.6872
Epoch [89/150], Loss: 1.7650
Epoch [90/150], Loss: 1.7229
Epoch [91/150], Loss: 1.7045
Epoch [92/150], Loss: 1.7145
Epoch [93/150], Loss: 1.6583
Epoch [94/150], Loss: 1.6762
Epoch [95/150], Loss: 1.7367
Epoch [96/150], Loss: 1.7172
Epoch [97/150], Loss: 1.7801
Epoch [98/150], Loss: 1.7196
Epoch [99/150], Loss: 1.7688
Epoch [100/150], Loss: 1.6751
Epoch [101/150], Loss: 1.7077
Epoch [102/150], Loss: 1.6897
Epoch [103/150], Loss: 1.6254
Epoch [104/150], Loss: 1.6416
Epoch [105/150], Loss: 1.7527
Epoch [106/150], Loss: 1.7265
Epoch [107/150], Loss: 1.7298
Epoch [108/150], Loss: 1.7873
Epoch [109/150], Loss: 1.7386
Epoch [110/150], Loss: 1.6844
Epoch [111/150], Loss: 1.7183
Epoch [112/150], Loss: 1.6143
Epoch [113/150], Loss: 1.7220
Epoch [114/150], Loss: 1.8364
Epoch [115/150], Loss: 1.765

Epoch [58/150], Loss: 1.9189
Epoch [59/150], Loss: 1.8668
Epoch [60/150], Loss: 1.9589
Epoch [61/150], Loss: 1.9740
Epoch [62/150], Loss: 1.8832
Epoch [63/150], Loss: 1.9242
Epoch [64/150], Loss: 1.8793
Epoch [65/150], Loss: 1.9293
Epoch [66/150], Loss: 2.0036
Epoch [67/150], Loss: 1.8607
Epoch [68/150], Loss: 1.9435
Epoch [69/150], Loss: 1.9343
Epoch [70/150], Loss: 1.9543
Epoch [71/150], Loss: 1.9478
Epoch [72/150], Loss: 1.8565
Epoch [73/150], Loss: 1.9155
Epoch [74/150], Loss: 1.8218
Epoch [75/150], Loss: 1.9270
Epoch [76/150], Loss: 1.8810
Epoch [77/150], Loss: 1.9036
Epoch [78/150], Loss: 1.9516
Epoch [79/150], Loss: 1.8255
Epoch [80/150], Loss: 1.8858
Epoch [81/150], Loss: 1.9271
Epoch [82/150], Loss: 1.8199
Epoch [83/150], Loss: 1.8997
Epoch [84/150], Loss: 1.9489
Epoch [85/150], Loss: 1.9552
Epoch [86/150], Loss: 1.9214
Epoch [87/150], Loss: 1.9205
Epoch [88/150], Loss: 1.9250
Epoch [89/150], Loss: 1.9321
Epoch [90/150], Loss: 1.9041
Epoch [91/150], Loss: 1.9355
Epoch [92/150]

KeyboardInterrupt: 

In [ ]:
#torch.save(cnn.state_dict(), 'cnn_a_long.pt')